[RFM 분석의 3요소]
- Recency: 얼마나 최근에 구매했는가 → 얼마나 최근에 판매했는가
- Frequency: 얼마나 자주 구매했는가 → 얼마나 자주 판매했는가
- Monetary: 얼마나 많이 지출했는가 → 얼마나 많이 판매했는가

[RFM 분석의 분류]
- 충성도(RFM의 종합적 고려)에 따른 분류
- 구매 빈도(F)에 따른 분류
- 지출금액(M)에 따른 분류
- 이외에도 주관적으로 분류 기준 수립이 가능
- 정해진 기준이 없어 자유롭게 분류 가능

IMaRD 양식 준수: 이전 강의자료 참조
- 적절한 도표 활용: 주요 내용 또는 강조할 내용은 도표가 필수
- 비교, 분류는 수치형 데이터를 이용
- 불명확한 데이터는 보고서에 넣지 않는다
- 내 논리와 반하는 데이터는 기본적으로 넣지 않는다
- 도표에 필요 없는 데이터는 삭제한다
- 분석 결과는 재현성이 보장되어야 한다
- 데이터 비교 기준 통일: 전처리 기준, 도표의 스케일 등
- 데이터 분석 결과, 이상치를 고려해야 한다

[필수 분석]
1. 입점 기업별 RFM 분석(3그룹) 진행

2. 매출 시각화
   a. 월별, 연도별 매출
   b. 월별 순수익 (처리 상황, 할부기간 고려)

3. 결제 방법에 따른 분석
   a. 결제 방법과 결제금액의 연관성 분석
   b. 결제방법은 맨 앞의 한가지만 사용한 것으로 간주

[선택 분석]
1. 매출 시각화 2
   a. 최대 매출 상품 3종류 집계
   b. 주문 연도에 따른 해당 상품의 매출 증감 분석

2. 연관성 분석
   a. 주문한 달과 판매금액의 상관관계 분석
   ※ 연도의 변화는 무시한다

해당 프로젝트는 정답을 상정하지 않았습니다
필수 분석은 강의자료 참고만으로 해결할 수 있습니다
코드의 복사, 검색 등을 허용합니다
보고서에는 논리, 결과, 의견을 기재합니다
코드를 포함해 분석 보고서 형태로 제출합니다

In [ ]:
!pip install statannot

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from statannot import add_stat_annotation
from datetime import datetime, timedelta

mpl.rc('font', family='NanumGothic')   #한글 깨짐 현상 처리
mpl.rc('axes', unicode_minus=False)   #마이너스 깨짐 현상 처리

In [ ]:
#데이터 불러오기

df = pd.read_excel('미니프로젝트-쇼핑몰 실습데이터.xlsx')

전처리

In [ ]:
# 업체명 결측치 확인
df['업체명'].unique()

In [ ]:
# 업체명 결측치 제거
df2 = df.copy()
df2 = df.dropna(subset = '업체명')

In [ ]:
# 업체명과 상품명에 후불이라는 정보 포함
df2[df2['업체명']=='지니 태블릿(후불집행)']

In [ ]:
# '지니 태블릿(후불집행)'의 '결제방법'을 모두 '후불'로 변경
df3 = df2.copy()
df3.loc[df3['업체명'] == '지니 태블릿(후불집행)', '결제방법'] = '후불'

In [ ]:
# '지니 태블릿'과 '지니 태블릿(후불집행)'을 모두 '지니'로 바꾸기
df3['업체명'] = df3['업체명'].replace({'지니 태블릿': '지니', '지니 태블릿(후불집행)': '지니'})

# 결과 출력
print(df3)

In [ ]:
df[df['업체명'] == '지니'].shape   # 지니 업체명 증가한 것 확인 

상품명 전처리

In [ ]:
# 상품명 결측치 확인
df3[df3.상품명.isnull()]

In [ ]:
# 삭제
df4 = df3.copy()
df4 = df4.dropna(subset = '상품명')

결제방법 전처리

In [ ]:
# 결측치 확인
df4[df4.결제방법.isnull()]

In [ ]:
# 삭제
df5 = df4.copy()
df5 = df5.dropna(subset = '결제방법')

In [ ]:
# 제조사 결측치 확인
df5 = df4.copy()
df5[df5.제조사.isnull()]

In [ ]:
# 주문수량 0인 건 삭제
df6 = df5.copy()
df6 = df6[df6['주문수량'] != 0]

In [ ]:
# 판매금액이 0인 건 삭제
df7 = df6.copy()
df7 = df7[df7['판매금액'] != 0]

제작문구 내역 처리

In [ ]:
# 대부분이 결측치
df7[df7['제작문구 내역'].isnull()]

In [ ]:
# 삭제
df8 = df7.copy()
df8.drop(['제작문구 내역'], axis = 1, inplace = True)

할부기간

In [ ]:
# 결측치 일시불로 변환
df8.할부기간.unique()

In [ ]:
df9 = df8.copy()
df9['할부기간'] = df9['할부기간'].fillna('일시불')

결제방법

In [ ]:
df9['결제방법'].unique()

In [ ]:
df9[df9['결제방법'] == '-'].처리상태.value_counts()

In [ ]:
# 결제방법이 '-' 인 경우 삭제
df10 = df9.copy()
df10 = df10[df10['결제방법'] != '-']

In [ ]:
# ' + 포'
# 주 결제방법을 알 수 없으므로 '-'와 마찬가지로 삭제
df11 = df10.copy()
df11 = df11[df11['결제방법'] != ' + 포']

In [ ]:
# 여러 결제 방법이 섞인 경우 첫 번째 결제 방법만 남기기

df12 = df11.copy()
df12['결제방법'] = df12['결제방법'].astype(str)
df12['결제방법'] = df12['결제방법'].apply(lambda x: x.split(' + ')[0] if ' + ' in x else x)

In [ ]:
# 줄임말 늘리기

df13 = df12.copy()
df13.loc[df13['결제방법'] == '신', '결제방법'] = '신용카드'
df13.loc[df13['결제방법'] == '적', '결제방법'] = '적립금'
df13.loc[df13['결제방법'] == '후', '결제방법'] = '후불'
df13.loc[df13['결제방법'] == '정', '결제방법'] = '정기결제'
df13.loc[df13['결제방법'] == '현', '결제방법'] = '현금간편결제'
df13.loc[df13['결제방법'] == '가', '결제방법'] = '가상계좌'
df13.loc[df13['결제방법'] == '무', '결제방법'] = '무통장입금'
df13.loc[df13['결제방법'] == '포', '결제방법'] = '포인트'
df13.loc[df13['결제방법'] == '웰', '결제방법'] = '웰컴마일'

전처리 완료

In [ ]:
df13.reset_index(drop=True)

1. 업체명으로 RFM 분석

In [ ]:
#판매금액을 기준으로 정렬하여 상위 업체 추출
상위업체 = df.groupby('업체명')['판매금액'].sum().reset_index()
상위업체 = 상위업체.sort_values(by='판매금액', ascending=False)

#판매금액이 많은 순으로 상위 3개 업체 출력
상위업체추출 = 상위업체.head(3)
print(상위업체추출)

R분석

In [ ]:
#주어진 데이터프레임에서 업체명이 '지니', '다우기술', '천재태블릿'에 해당하는 행만 추출
selected_companies = ['지니', '다우기술', '천재태블릿']
filtered_data = df[df['업체명'].isin(selected_companies)]

#2019~2020 / 2021 / 2022 3그룹으로 분류
df['월별'] = df['주문일자'].apply(lambda x:str(x).split('-')[1]) # 주문일자 월별 컬럼 추가
df['연도별'] = df['주문일자'].apply(lambda x:str(x).split('-')[0]) # 주문일자 연도별 컬럼 추가

df['연도별'].unique()

In [ ]:
def year(x):
    if x == '2022':
        return 3 # 가장 높은 점수 3점
    elif x == '2021':
        return 2 # 중간 점수 2점
    else:
        return 1 # 가장 낮은 점수 1점

df['R그룹'] = df['연도별'].apply(lambda x:year(x))

In [ ]:
# 업체명과 R값
r = ('지니', '다우기술', '천재태블릿')
df['R'] = df['업체명'].apply(lambda x: x in r)
df_r = df[df['R'] == True]

# 파이차트 생성
fig = px.pie(df_r, values='R그룹', names='업체명',
             title='입점 기업별 R분석(3그룹)', color_discrete_sequence=['#ff9999', '#ffc000', '#8fd9b6'])

# 레이아웃 수정
fig.update_layout(
    title=dict(text='입점 기업별 R분석(3그룹)', x=0.5),  # 제목 가운데로 이동
    autosize=False,  # 자동 크기 조절 비활성화
    width=600,       # 가로 크기
    height=600       # 세로 크기
)

fig.show()

F분석

In [ ]:
#주어진 데이터프레임에서 업체명이 '지니', '다우기술', '천재태블릿'에 해당하는 행만 추출
F그룹 = ['지니', '다우기술', '천재태블릿']
filtered_data = df[df['업체명'].isin(selected_companies)]

df['주문수량'] = df['주문수량'].apply(lambda x:str(x).split(',')[0])
df['주문수량'] = df['주문수량'].apply(lambda x:int(x))

df['주문수량'].unique()

In [ ]:
def fre(x):
    if x > 100:
        return 3 # 가장 높은 점수 3점
    elif x > 50:
        return 2 # 중간 점수 2점
    else:
        return 1 # 가장 낮은 점수 1점
    
df['F그룹'] = df['주문수량'].apply(lambda x:fre(x))

In [ ]:
# 업체명과 F값
r = ('지니', '다우기술', '천재태블릿')
df['F'] = df['업체명'].apply(lambda x: x in r)
df_r = df[df['F'] == True]

# 파이차트 생성
fig = px.pie(df_r, values='F그룹', names='업체명',
             title='입점 기업별 F분석(3그룹)', color_discrete_sequence=['#ff9999', '#ffc000', '#8fd9b6'])

# 레이아웃 수정
fig.update_layout(
    title=dict(text='입점 기업별 F분석(3그룹)', x=0.5),  # 제목 가운데로 이동
    autosize=False,  # 자동 크기 조절 비활성화
    width=600,       # 가로 크기
    height=600       # 세로 크기
)

fig.show()

M분석

In [ ]:
# 주문수량, 판매금액 데이터 인트형으로 변환
df['판매금액'] = df['판매금액'].astype(str).str.replace(',', '').astype(int)
df['주문수량'] = df['주문수량'].astype(str).str.replace(',', '').astype(int)

df['매출'] = df['주문수량'] * df['판매금액']

# 업체명이 '지니', '다우기술', '천재태블릿'에 해당하는 행만 추출
selected_companies = ['지니', '다우기술', '천재태블릿']
filtered_data = df[df['업체명'].isin(selected_companies)]

In [ ]:
df['판매금액'].describe()

In [ ]:
def pay(x):
    if x > 2.790000e+04:
        return 3 # 가장 높은 점수 3점
    elif x > 9.600000e+03:
        return 2 # 중간 점수 2점
    else:
        return 1 # 가장 낮은 점수 1점

In [ ]:
df['M그룹'] = df['판매금액'].apply(lambda x:pay(x))

In [ ]:
# 업체명과 M값
r = ('지니', '다우기술', '천재태블릿')
df['M'] = df['업체명'].apply(lambda x: x in r)
df_r = df[df['M'] == True]

# 파이차트 생성
fig = px.pie(df_r, values='M그룹', names='업체명',
             title='입점 기업별 M분석(3그룹)', color_discrete_sequence=['#ff9999', '#ffc000', '#8fd9b6'])

# 레이아웃 수정
fig.update_layout(
    title=dict(text='입점 기업별 M분석(3그룹)', x=0.5),  # 제목 가운데로 이동
    autosize=False,  # 자동 크기 조절 비활성화
    width=600,       # 가로 크기
    height=600       # 세로 크기
)

fig.show()

2. 매출 시각화
   a. 월별, 연도별 매출

In [ ]:
# '주문일자'를 datetime type으로 변환
df['주문일자'] = pd.to_datetime(df['주문일자'])

# '주문일자'에서 년도 추출
df['년도'] = df['주문일자'].dt.year

# '년도'끼리 묶고 각 년도의 판매금액 총합 구하기
df20 = total_sales_by_year = df.groupby('년도')['판매금액'].sum().reset_index()

print(total_sales_by_year)

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'

# 시각화
plt.figure(figsize=(10, 6))

# 선 그래프 그리기
plt.figure(figsize=(10, 6))

# 선 그래프 그리기
sns.lineplot(data=total_sales_by_year, x='년도', y='판매금액', marker='o', color='skyblue', linewidth=2)

# 그래프 스타일 설정
sns.set(style="whitegrid")

# x축, y축 레이블 설정
plt.title('연도별 매출', fontsize=16)
plt.xlabel('년도', fontsize=14)
plt.ylabel('총 판매금액', fontsize=14)

# x축 값 설정
plt.xticks(total_sales_by_year['년도'])

# 그리드 눈금선 추가
plt.grid(True, linestyle='--', alpha=0.7)

# 그래프에 값 표시

plt.show()

In [ ]:
#'월' 열을 추출
df['월'] = df['주문일자'].dt.month

#'월'을 기준으로 그룹화하고 판매금액을 합산
total_sales_by_month = df.groupby(['월'])['판매금액'].sum().reset_index()

# 결과 출력
print(total_sales_by_month)

In [ ]:
from matplotlib.ticker import MaxNLocator  # Import MaxNLocator
plt.rcParams['font.family'] = 'Malgun Gothic'

# 시각화
plt.figure(figsize=(10, 6))

# 선 그래프 그리기
sns.lineplot(data=total_sales_by_month, x='월', y='판매금액', marker='o', color='skyblue')

# 그래프 스타일 설정
sns.set(style="whitegrid")

# x축, y축 레이블 설정
plt.title('월별 매출')
plt.xlabel('월')
plt.ylabel('총 판매금액')

# 그리드 눈금선 추가
plt.grid(True, linestyle='--', alpha=0.7)

# x축 간격을 1로 설정
plt.xticks(range(1, 13))  # x축의 눈금을 1부터 12까지의 정수로 설정

plt.show()

   b. 월별 순수익(처리 상황 고려)

In [ ]:
df['처리상태'].unique() 

In [ ]:
a = ('구매확정')
df['구매'] = df['처리상태'].apply(lambda x:x in a)
df14 = df[df['구매'] == True]

In [ ]:
sns.lineplot(data=df14, x='월별', y='매출')

plt.show()

   b. 월별 순수익(처리 상황+할부기간 고려)

In [ ]:
shop = df13.copy().reset_index(drop = True)

In [ ]:
shop_full = shop[shop['할부기간'] == '일시불'] # 일시불 df

In [ ]:
shop_fullym = shop_full.copy()
shop_fullym['주문일자(연월)'] = shop_full['주문일자'].dt.strftime('%Y-%m') # 연도-월 추출

In [ ]:
shop_fullym1 = shop_fullym.copy()
shop_fullym1 = shop_fullym[['주문일자(연월)','판매금액']] # 연월 컬럼과 판매금액 컬럼만 따로 저장

In [ ]:
shop_fullym2 = shop_fullym1.groupby(['주문일자(연월)'], as_index=False).sum() # 연월 컬럼 그룹으로 묶어 판매금액 합산

In [ ]:
shop_fullym2['주문일자(연월)'] = pd.to_datetime(shop_fullym2['주문일자(연월)']) # 연월 컬럼을 다시 날짜 형식으로

In [ ]:
shop_fullym3 = shop_fullym2.set_index('주문일자(연월)') # 연월 컬럼을 인덱스로

In [ ]:
shop_fullym3.head() # 결과 확인

반복

In [ ]:
# 할부 24개월
shop_24 = shop[shop['할부기간'] == '24개월']

In [ ]:
shop_24ym = shop_24.copy()
shop_24ym['주문일자(연월)'] = shop_24['주문일자'].dt.strftime('%Y-%m')
shop_24ym1 = shop_24ym.copy()
shop_24ym1 = shop_24ym[['주문일자(연월)','판매금액']]
shop_24ym2 = shop_24ym1.groupby(['주문일자(연월)'], as_index=False).sum()
shop_24ym2['주문일자(연월)'] = pd.to_datetime(shop_24ym2['주문일자(연월)'])
shop_24ym3 = shop_24ym2.set_index('주문일자(연월)')
shop_24ym4 = shop_24ym3.copy()
shop_24ym4.head()

In [ ]:
# 할부 18개월
shop_18 = shop[shop['할부기간'] == '18개월']

In [ ]:
shop_18ym = shop_18.copy()
shop_18ym['주문일자(연월)'] = shop_18['주문일자'].dt.strftime('%Y-%m')
shop_18ym1 = shop_18ym.copy()
shop_18ym1 = shop_18ym[['주문일자(연월)','판매금액']]
shop_18ym2 = shop_18ym1.groupby(['주문일자(연월)'], as_index=False).sum()
shop_18ym2['주문일자(연월)'] = pd.to_datetime(shop_18ym2['주문일자(연월)'])
shop_18ym3 = shop_18ym2.set_index('주문일자(연월)')
shop_18ym4 = shop_18ym3.copy()
shop_18ym4.head()

In [ ]:
# 할부 12개월
shop_12 = shop[shop['할부기간'] == '12개월']

In [ ]:
shop_12ym = shop_12.copy()
shop_12ym['주문일자(연월)'] = shop_12['주문일자'].dt.strftime('%Y-%m')
shop_12ym1 = shop_12ym.copy()
shop_12ym1 = shop_12ym[['주문일자(연월)','판매금액']]
shop_12ym2 = shop_12ym1.groupby(['주문일자(연월)'], as_index=False).sum()
shop_12ym2
shop_12ym2['주문일자(연월)'] = pd.to_datetime(shop_12ym2['주문일자(연월)'])
shop_12ym3 = shop_12ym2.set_index('주문일자(연월)')
shop_12ym4 = shop_12ym3.copy()
shop_12ym4.head()

In [ ]:
# 할부 6개월
shop_6 = shop[shop['할부기간'] == '6개월']

In [ ]:
shop_6ym = shop_6.copy()
shop_6ym['주문일자(연월)'] = shop_6['주문일자'].dt.strftime('%Y-%m')
shop_6ym1 = shop_6ym.copy()
shop_6ym1 = shop_6ym[['주문일자(연월)','판매금액']]
shop_6ym2 = shop_6ym1.groupby(['주문일자(연월)'], as_index=False).sum()
shop_6ym2
shop_6ym2['주문일자(연월)'] = pd.to_datetime(shop_6ym2['주문일자(연월)'])
shop_6ym3 = shop_6ym2.set_index('주문일자(연월)')
shop_6ym4 = shop_6ym3.copy()
shop_6ym4.head()

In [ ]:
# 할부 1개월
shop_1m = shop[shop['할부기간'] == '1개월']

In [ ]:
shop_1mym = shop_1m.copy()
shop_1mym['주문일자(연월)'] = shop_1m['주문일자'].dt.strftime('%Y-%m')
shop_1mym1 = shop_1mym.copy()
shop_1mym1 = shop_1mym[['주문일자(연월)','판매금액']]
shop_1mym2 = shop_1mym1.groupby(['주문일자(연월)'], as_index=False).sum()
shop_1mym2
shop_1mym2['주문일자(연월)'] = pd.to_datetime(shop_1mym2['주문일자(연월)'])
shop_1mym3 = shop_1mym2.set_index('주문일자(연월)')
shop_1mym4 = shop_1mym3.copy()
shop_1mym4.head()

In [ ]:
#판매금액을 각 할부개월로 나눈 뒤 1달씩 더해가며 할부개월 수 만큼 일시불 df에 더하기

def installment24(y):
    global shop_fullym3 # -> 일시불 df
    shop_24ym4['판매금액'] /= y # 판매금액을 할부개월수로 나눔
    for i in range(1, y + 1): # 할부개월 수 만큼 반복
        # 1달씩 더함. timedelta에 달을 더하는 기능이 없어 31일로 대체.며칠 씩 더 더해도 2년 안에 한 달이 바뀌지는 않음.
        shop_24ym4.index = pd.DatetimeIndex(shop_24ym4.index) + timedelta(days=31)
        shop_fullym3 = shop_fullym3.add(shop_24ym4, fill_value=0) # 일시불 df에 더함.

In [ ]:
installment24(24)

In [ ]:
# 반복
def installment18(y):
    global shop_fullym3
    shop_18ym4['판매금액'] /= y
    for i in range(1, y + 1):
        shop_18ym4.index = pd.DatetimeIndex(shop_18ym4.index) + timedelta(days=31)
        shop_fullym3 = shop_fullym3.add(shop_18ym4, fill_value=0)

In [ ]:
installment18(18)

In [ ]:
def installment12(y):
    global shop_fullym3
    shop_12ym4['판매금액'] /= y
    for i in range(1, y + 1):
        shop_12ym4.index = pd.DatetimeIndex(shop_12ym4.index) + timedelta(days=31)
        shop_fullym3 = shop_fullym3.add(shop_12ym4, fill_value=0)

In [ ]:
installment12(12)

In [ ]:
def installment6(y):
    global shop_fullym3
    shop_6ym4['판매금액'] /= y
    for i in range(1, y + 1):
        shop_6ym4.index = pd.DatetimeIndex(shop_6ym4.index) + timedelta(days=31)
        shop_fullym3 = shop_fullym3.add(shop_6ym4, fill_value=0)

In [ ]:
installment6(6)

In [ ]:
def installment1m(y):
    global shop_fullym3
    shop_1mym4['판매금액'] /= y
    for i in range(1, y + 1):
        shop_1mym4.index = pd.DatetimeIndex(shop_1mym4.index) + timedelta(days=31)
        shop_fullym3 = shop_fullym3.add(shop_1mym4, fill_value=0)

In [ ]:
installment1m(1)

In [ ]:
# 할부금이 각 달에 배정된 총 판매금액
shop_install = shop_fullym3.reset_index(drop=False)
shop_install

In [ ]:
# 2022년 11월 이후 지우기
shop_install2 = shop_install[shop_install['주문일자(연월)'] < '2022-12-01']

In [ ]:
# '주문일자(연월)' 열을 datetime 형식으로 변환
shop_install2['주문일자(연월)'] = pd.to_datetime(shop_install2['주문일자(연월)'])

In [ ]:
# 'day'를 제외하고 'month' 열을 생성
shop_install_m = shop_install2.copy()
shop_install_m['month'] = shop_install_m['주문일자(연월)'].dt.to_period("M")

In [ ]:
shop_install_m

In [ ]:
# 'month' 열을 기준으로 그룹화하고 판매금액을 합산
shop_install_m2 = shop_install_m.groupby('month')['판매금액'].sum().reset_index()

In [ ]:
# xtick이 생략돼서 나와서 문자열로 고치기

shop_install_m3 = shop_install_m2.copy()
shop_install_m3['month'] = shop_install_m2['month'].apply(lambda x: str(x))

In [ ]:
# 할부기간 고려 월별 판매금액 그래프

import plotly.express as px

# 억 원으로 판매금액 단위 변경
shop_install_m3['판매금액(억원)'] = shop_install_m3['판매금액'] / 1e8

fig = px.bar(shop_install_m3, x='month', y='판매금액(억원)',
             labels={'판매금액(억원)': '판매금액(억원)', 'month': ''})
fig.update_traces(marker_color='rgb(158,202,225)')

# 그래프에 제목 추가
fig.update_layout(title_text='월별 판매금액 그래프', title_x=0.5)

fig.show()

3. 결제 방법에 따른 분석

In [ ]:
pay = df13.copy()

In [ ]:
# 1% 표본 뽑아서 분석

pay_rn = pay.sample(frac = 0.01)

In [ ]:
plt.rcParams['font.family'] = 'Malgun Gothic'

sns.scatterplot(x = pay_rn['결제방법'], y = pay_rn['판매금액'])
plt.ylabel('판매금액(천만 원)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 가설: 신용 거래가 현금 거래보다 거래금액이 클 것이다.

In [ ]:
sns.kdeplot(pay_rn['판매금액'])

In [ ]:
pay2 = pay_rn.copy()
pay2['판매금액_log2'] = np.log2(pay_rn['판매금액'])

In [ ]:
sns.kdeplot(pay2['판매금액_log2'])

In [ ]:
# shapiro pvalue=1.476911539869839e-23 -> 정규성 만족 x

stats.shapiro(pay2['판매금액_log2'])

In [ ]:
# 신용카드, 후불, 정기결제만 따로 payt에 저장

payt = pay2.copy()

target1 = ['신용카드', '후불', '정기결제']
payt = pay2[pay2['결제방법'].apply(lambda x: True if x in target1 else False)]

In [ ]:
payt.결제방법.unique()

In [ ]:
# payt의 신용여부 열에는 '신용' 표시

payt['신용여부'] = '신용'

In [ ]:
# 현금, 포인트, 마일리지 등은 payc 에 저장

payc = pay2[pay2['결제방법'].apply(lambda x: False if x in target1 else True)]

In [ ]:
# payc의 신용여부 컬럼에는 '현금성' 표시

payc['신용여부'] = '현금성'

In [ ]:
sns.kdeplot(payt['판매금액_log2'])
sns.kdeplot(payc['판매금액_log2'])

In [ ]:
# 신용 현금 합치기

payct = pd.concat([payt, payc])

In [ ]:
# 정규성 만족 안하므로 Mann-Whitney
# pvalue=2.1255309268239036e-07

stats.mannwhitneyu(payt['판매금액_log2'], payc['판매금액_log2'])

In [ ]:
# D: 입력할 데이터프레임 이름
# col: x축 column 이름
# val: y축 value 이름
# group1, group2: col 기준으로 분류된 이름

from statannot import add_stat_annotation

def auto_test(D, col, val, group1, group2):
    f1 = sns.boxplot(data = D, x = col, y = val)
    f2 = sns.stripplot(data = D, x = col, y = val, color='k', alpha=0.1)
    
  
    add_stat_annotation(f1, data = D, x = col, y = val,
                        box_pairs=[((group1),(group2))], # [(().())] 형식
                        test='Mann-Whitney', text_format='star', # 정규성 만족 안하므로 Mann-Whitney
                        loc='inside', verbose=2) # verbose는 기준 출력.
 
   
    plt.xlabel('')
    plt.ylabel('')
        
    plt.show()

In [ ]:
# P_val=2.126e-07

auto_test(payct, '신용여부', '판매금액_log2', '신용', '현금성')